In [16]:
print("Validation:")
print(f"Original rows: {pd.read_csv('/content/Warehouse_and_Retail_Sales.csv').shape[0]}")
print(f"Cleaned rows: {df.shape[0]}")
print(f"Suppliers: {suppliers.shape[0]}, Items: {items.shape[0]}, Monthly Movements: {monthly_movements.shape[0]}")

Validation:
Original rows: 307645
Cleaned rows: 307645
Suppliers: 397, Items: 35165, Monthly Movements: 307645


In [14]:
import sqlite3
os.makedirs('output', exist_ok=True)

conn = sqlite3.connect('output/etl_output.db')

suppliers.to_sql('suppliers', conn, if_exists='replace', index=False)
items.to_sql('items', conn, if_exists='replace', index=False)
monthly_movements.to_sql('monthly_movements', conn, if_exists='replace', index=False)

conn.close()

In [13]:
import os
os.makedirs('processed', exist_ok=True)

suppliers.to_csv('processed/suppliers.csv', index=False)
items.to_csv('processed/items.csv', index=False)
monthly_movements.to_csv('processed/monthly_movements.csv', index=False)

In [12]:
# Suppliers table
suppliers = df[['supplier']].drop_duplicates()

# Items table
items = df[['item_code', 'item_description', 'item_type']].drop_duplicates()

# Monthly Movements table
monthly_movements = df[['year', 'month', 'item_code', 'supplier',
                        'retail_sales', 'retail_transfers', 'warehouse_sales',
                        'total_movement', 'high_movement']]

In [11]:
# Total movement = sum of all sales/transfers
df['total_movement'] = df['retail_sales'] + df['retail_transfers'] + df['warehouse_sales']

# Example segment flag: high movement items
df['high_movement'] = df['total_movement'] > 50  # adjust threshold as needed

In [8]:
# Margin
if 'price' in df.columns and 'cost' in df.columns:
    df['margin'] = df['price'] - df['cost']

# Example segment flags (adjust thresholds as needed)
if 'quantity' in df.columns:
    df['high_volume_order'] = df['quantity'] > 50


In [7]:
# Lowercase, replace spaces with underscores
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Convert types
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)
df['item_code'] = df['item_code'].astype(str)

In [3]:
# Drop duplicates
df = df.drop_duplicates()

# Handle missing values (simple example)
df = df.fillna({
    'price': 0,
    'quantity': 0,
    'customer_segment': 'Unknown'
})

print(f"Dataset shape after cleaning: {df.shape}")

Dataset shape after cleaning: (307645, 9)


In [2]:
df = pd.read_csv('/content/Warehouse_and_Retail_Sales.csv')

In [1]:
import pandas as pd
import os
